In [2]:
import numpy as np

# Schwefel function
$$ f(x) = 418.9829n - \sum_{i=1}^{n}x_i \sin(\sqrt{|x_i|}) $$

In [37]:
def schwefel_function(x):
    n = len(x)
    a = 418.9829
    sum_term = np.sum(x * np.sin(np.sqrt(np.abs(x))))
    result = a * n - sum_term
    return result

In [63]:
dimension = 5
range_x = [-500, 500]
random_vector = np.random.uniform(range_x, dimension)
random_vector = np.array([420.9687,420.9687,420.9687])
result = schwefel_function(random_vector)
print(f"Schwefel function value for {random_vector}: {result}")

Schwefel function value for [420.9687 420.9687 420.9687]: 3.818351251538843e-05


# Ackley function
$$ f(x) = -20 \exp\left(-0.2 \sqrt{\frac{1}{n} \sum_{i=1}^{n}x_i^2}\right) - \exp\left(\frac{1}{n}\sum_{i=1}^{n}\cos(2\pi x_i)\right) + 20 + e $$


In [55]:
def ackley_function(x):
    n = len(x)
    a = 20
    b = 0.2
    c = 2 * np.pi
    term1 = -a * np.exp(-b * np.sqrt((1/n) * np.sum(x**2)))
    term2 = -np.exp((1/n) * np.sum(np.cos(c * x)))
    result = term1 + term2 + 20 + np.exp(1)
    return result

In [57]:
dimension = 5
range_x = [-32.768, 32.768]
# random_vector = np.random.uniform(range_x, dimension)
random_vector = np.array([0,0])
result = ackley_function(random_vector)
print(f"Ackley function value for {random_vector}: {result}")

Ackley function value for [0 0]: 4.440892098500626e-16


# Evolutionary Startegy

Ackley Function:

    Diversity and Exploration:
        The Ackley function, with its single, wide global minimum surrounded by many local minima, benefits from exploration to escape local optima.
        Discrete recombination, by introducing diversity within the phenotype space, may be beneficial for exploring different regions of the search space.

    Strategy Parameters:
        Since the Ackley function requires a balance between exploration and exploitation, a combination of discrete recombination for the object variable part and intermediate recombination for strategy parameters may be a reasonable choice.

Schwefel Function:
    
    Multimodality and Local Minima:
        The Schwefel function, being highly multimodal with many local minima, requires effective exploration to discover the global minimum.
        Global recombination, involving more than two parents and allowing for diverse combinations, may help the algorithm escape local minima and explore the complex landscape.

    Adaptation of Strategy Parameters:
        The rugged landscape of the Schwefel function suggests that cautious adaptation of strategy parameters is important.
        Intermediate recombination for strategy parameters can contribute to a more cautious adjustment of mutation or recombination step sizes.



In [ ]:
def population_initialization():
    

In [67]:
# Randomly choose one parent for each gene
def global_discrete_recombination(population, num_parents_per_gene=2):
    num_genes = len(population[0])
    selected_parents = np.random.choice(population, size=(num_genes, num_parents_per_gene), replace=True)
    offspring = np.array([np.random.choice(selected_parents[gene]) for gene in range(num_genes)])
    return offspring

In [65]:
# Randomly select genes from either parent
def local_discrete_recombination(parent1, parent2):
    offspring = np.zeros_like(parent1)
    for i in range(len(offspring)):
        if np.random.rand() < 0.5:
            offspring[i] = parent1[i]
        else:
            offspring[i] = parent2[i]
    return offspring

In [66]:
# Taking the average
def local_intermediary_recombination(parent1, parent2):
    offspring = (parent1 + parent2) / 2.0 
    return offspring

In [68]:
# Take the average of Randomly select parents for each gene
def global_intermediary_recombination(population, num_parents_per_gene=2):
    num_genes = len(population[0])
    selected_parents = np.random.choice(population, size=(num_genes, num_parents_per_gene), replace=True)
    offspring = np.mean(selected_parents, axis=1)
    return offspring

$$ \sigma'_i = \sigma_i \cdot \exp\left(\tau' \cdot \mathcal{N}(0,1) + \tau \cdot \mathcal{N_i}(0,1)\right) $$

$$ x'_i = x_i + \sigma'_i \cdot \mathcal{N}(0,1) $$

$ \tau $ is proportional to $ \frac{1}{\sqrt{2n}} $

$ \tau' $ is proportional to $ \frac{1}{\sqrt{2\sqrt{n}}} $

$ \mathcal{N}(0,1) $ represents a random value drawn from a standard normal distribution,

$ \sigma'_i < \epsilon_0 $ implies $ \sigma'_i = \epsilon_0 $.



In [74]:
def uncorrelated_mutation_n_sigma(chromosome, n_sigma, epsilon):
    random_values = np.random.randn(len(chromosome))
    tau = 1 / np.sqrt(2 * len(chromosome))
    tau_prime = 1 / np.sqrt(2 * np.sqrt(len(chromosome)))
    new_n_sigma = n_sigma * np.exp(tau_prime * np.random.randn() + tau * random_values)
    mask = new_n_sigma < epsilon
    n_sigma[mask] = new_n_sigma[mask]
    random_values = np.random.randn(len(chromosome))
    mutated_chromosome = chromosome + n_sigma * random_values
    return mutated_chromosome, n_sigma

$$ \sigma' = \sigma \cdot \exp\left(\tau \cdot \mathcal{N}(0,1)\right) $$

$$ x'_i = x_i + \sigma' \cdot \mathcal{N}(0,1) $$

$ \tau $ is proportional to $ \frac{1}{\sqrt{n}} $

$ \mathcal{N}(0,1) $ represents a random value drawn from a standard normal distribution,

$ \sigma' < \epsilon_0 $ implies $ \sigma' = \epsilon_0 $.



In [75]:
def uncorrelated_mutation_one_sigma(chromosome, one_sigma, epsilon):
    random_value = np.random.randn()
    tau = 1 / np.sqrt(len(chromosome))
    one_sigma *= np.exp(tau * random_value)
    random_value = np.random.randn()
    mutated_chromosome = chromosome + one_sigma * random_value
    return mutated_chromosome, one_sigma

In [76]:
def elitism_survival_selection(parents, offsprings, objective_function):
    combined_population = np.vstack((parents, offsprings))
    new_generation_indices = np.argsort(objective_function(combined_population))[:len(parents)]
    new_generation = combined_population[new_generation_indices]
    return new_generation

In [77]:
def genrational_survival_selection(parents, offsprings, objective_function):
    new_generation = np.sort(objective_function(offsprings))[:len(parents)]
    return new_generation